In [21]:
import handcalcs.render

# Rentesregning, annuitetsopsparing og -gæld

## Renteomregning - år til måned

$$r_m = (1+r_å)^{1/12} - 1 $$

## Renteomregning - år til kvartal

$$r_k = (1+r_å)^{1/4} - 1 $$

### Eksempel

Bestem den kvartalsvise rente, hvis den årlige rente er $3\%$

$$r_k = (1+0,03)^{1/4} - 1 = 0,0074$$

dvs, den kvartalsvise rente er $$0,74\%$$

## Annuitetsopsparing

Hvis man indbetaler det samme beløb $b$ hver termin, og rentefoden er $r$, så er kapitalen $A$ umiddelbart efter den $n$’te indbetaling:

$$A = b\frac{(1 + r )^n - 1 }{r}$$

### Eksempel

En familie vil spare op til deres drømmehus. De vil hver måned indbetale 6500 kr. Den månedlige rente er $0,25\%$. Bestem saldoen på familiens opsparing efter 10 år

$$ A = 6500 \frac{(1+0,0025)^{120} - 1}{0,0025} = 908319,22$$

## Gældsannuitet

Hvis man indbetaler ydelsen y hver termin i n terminer (første gang man betaler er én termin efter lånet er optaget), og rentefoden er r , så er lånets størrelse:

$$G =  y \frac{1-(1+r)^{-n}}{r}$$

### Eksempel

En familie har råd til at betale et lån til en bolig tilbage med en ydelse på 20000 kr pr kvartal gennem 30 år. Bestem hvor stort et beløb familien kan låne, når den kvartalsvise rente er 0,74%

$$G = 20000 \frac{1-(1,0074)^{-120}}{0,0074} = 1586958,36$$

In [17]:
20000*4*30 - 1586958.36

813041.6399999999

### Henvisninger

https://www.webmatematik.dk/lektioner/matematik-c/funktioner/opsparingsannuitet

https://www.webmatematik.dk/lektioner/matematik-c/funktioner/galdsannuitet

In [28]:
%%render
r_å = 0.03
r_k = (1+0.03)**(1/4)-1

<IPython.core.display.Latex object>